In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import random
import os

# Load dataset (Extended handwritten-like text)
text = """The quick brown fox jumps over the lazy dog.
Handwriting generation is fun! AI can create amazing things.
This is an example of a character-level RNN learning patterns."""

# Create character mapping
chars = sorted(list(set(text)))
char_to_index = {c: i for i, c in enumerate(chars)}
index_to_char = {i: c for i, c in enumerate(chars)}

# Prepare sequences
sequence_length = 10
dataX = []
dataY = []
for i in range(len(text) - sequence_length):
    seq_in = text[i:i + sequence_length]
    seq_out = text[i + sequence_length]
    dataX.append([char_to_index[char] for char in seq_in])
    dataY.append(char_to_index[seq_out])

# Reshape and normalize
dataX = np.array(dataX)
dataY = to_categorical(dataY, num_classes=len(chars))

# Build the RNN model
model = Sequential([
    Embedding(len(chars), 8, input_length=sequence_length),
    LSTM(128, return_sequences=True),
    LSTM(128),
    Dense(len(chars), activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(dataX, dataY, epochs=100, batch_size=8)

# Function for temperature-based text generation
def sample_with_temperature(preds, temperature=1.0):
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds + 1e-8) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# Generate text with temperature
def generate_text(seed_text, length=200, temperature=0.8):
    result = seed_text
    for _ in range(length):
        x_input = np.array([[char_to_index[char] for char in result[-sequence_length:]]])
        x_input = pad_sequences(x_input, maxlen=sequence_length)
        prediction = model.predict(x_input, verbose=0)[0]
        next_index = sample_with_temperature(prediction, temperature)
        next_char = index_to_char[next_index]
        result += next_char
    return result

# Generate and print text
print("Generated Text:")
print(generate_text("Handwriting", length=200, temperature=0.8))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - accuracy: 0.0964 - loss: 3.5849
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.1572 - loss: 3.2796
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.1279 - loss: 3.1959
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.1223 - loss: 3.1333
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.1666 - loss: 3.1789
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.1478 - loss: 3.1435
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.1688 - loss: 3.0387
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.1576 - loss: 3.0559
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.1730 - loss: 3.0887
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.1900 - loss: 3.0275
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.1620 - loss: 3.1033
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step